In [1]:
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.4/julia-1.4.1-linux-x86_64.tar.gz' -O julia.tar.gz
    tar -x -f julia.tar.gz -C /usr/local --strip-components 1
    rm julia.tar.gz
fi
    julia -e '\
        using Pkg                                                          ;\
        Pkg.update()                                                       ;\
        Pkg.add("IJulia")                                                  ;\
        Pkg.add("Flux")                                                    ;\
        Pkg.add("MLDatasets")                                              ;\
        Pkg.add("Statistics")                                              ;\
        Pkg.update()                                                       ;\
        Pkg.precompile()                                                   ;\'
echo 'Done'

   Updating registry at `~/.julia/registries/General`
   Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
   Updating registry at `~/.julia/registries/General`
   Updati

# Julia 深度學習：類神經網路模型簡介

## 作業 032：訓練 MLP 學習門牌號碼資料集

訓練一個 MLP 模型來學習門牌號碼資料集。

注意：MLP 模型的能力有限，可能會導致訓練起來效果不佳。

注意：近期 Flux 正在持續更新，請確保您的 Julia 在 v1.3 版以上，以及 Flux 在 v0.10.4 以上或是最新版。

## 讀取資料

呼叫 SVHN2 資料集的過程中，會先去檢查以前是否有下載過，如果是第一次下載，過程中會詢問是否下載資料集，請回答 `y`。整個資料集的大小約為 1.6 GB，下載時間可能會稍久，請耐心等候。


    using Flux
    using Flux.Data: DataLoader
    using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
    using MLDatasets
    using Statistics

    train_X, train_y = SVHN2.traindata(Float32)
    test_X,  test_y  = SVHN2.testdata(Float32)

    train_X = Flux.flatten(train_X)
    test_X = Flux.flatten(test_X)
    train_y = onehotbatch(train_y, 1:10)
    test_y = onehotbatch(test_y, 1:10)

In [4]:
!julia -e '\
using Flux                                                                                        ;\
using Flux.Data: DataLoader                                                                       ;\
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy                            ;\
using MLDatasets                                                                                  ;\
using Statistics                                                                                  ;\
train_X, train_y = SVHN2.traindata(Float32)                                                       ;\
test_X,  test_y  = SVHN2.testdata(Float32)                                                        ;\
train_X = Flux.flatten(train_X)                                                                   ;\
test_X = Flux.flatten(test_X)                                                                     ;\
train_y = onehotbatch(train_y, 1:10)                                                              ;\
test_y = onehotbatch(test_y, 1:10)                                                                ;\
batchsize = 1024                                                                                  ;\
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true)                           ;\
test = DataLoader(test_X, test_y, batchsize=batchsize)                                            ;\
model = Chain( Dense(3072, 512, relu), Dense(512, 64, relu),  Dense(64, 10),  softmax )           ;\
loss(x, y) = logitcrossentropy(model(x), y)                                                       ;\
function test_loss()                                                                              ;\
    l = 0f0                                                                                       ;\
    for (x, y) in test                                                                            ;\
        l += loss(x, y)                                                                           ;\
    end                                                                                           ;\
    l/length(test)                                                                                ;\
end                                                                                               ;\
evalcb() = @show(test_loss())                                                                     ;\
epochs = 10                                                                                       ;\
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.005), cb=throttle(evalcb, 10))      ;\
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))                                           ;\
println( )                                                                                        ;\
println( "accuracy :" , accuracy(test_X, test_y) )                                                ;\'

[ Info: Epoch 1
test_loss() = 2.350677f0
test_loss() = 2.3023286f0
[ Info: Epoch 2
test_loss() = 2.3023286f0
test_loss() = 2.3023286f0
[ Info: Epoch 3
test_loss() = 2.3023286f0
test_loss() = 2.3023286f0
[ Info: Epoch 4
test_loss() = 2.3023286f0
test_loss() = 2.3023286f0
[ Info: Epoch 5
test_loss() = 2.3023286f0
test_loss() = 2.3023286f0
[ Info: Epoch 6
test_loss() = 2.3023286f0
test_loss() = 2.3023286f0
[ Info: Epoch 7
test_loss() = 2.3023286f0
test_loss() = 2.3023286f0
[ Info: Epoch 8
test_loss() = 2.3023286f0
test_loss() = 2.3023286f0
[ Info: Epoch 9
test_loss() = 2.3023286f0
test_loss() = 2.3023286f0
[ Info: Epoch 10
test_loss() = 2.3023286f0
test_loss() = 2.3023286f0

accuracy :0.15938076213890595
